# The redshift evolution of the BBH merger rate: "a weighty matter"

## Notebook to reproduce values that are quoted in the text of the paper



In [2]:
######################################
## Imports
import numpy as np
import h5py as h5

from astropy.table import Table, Column
import astropy.units as u
from astropy import constants as const

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import ticker, cm

from scipy import stats

# Chosen cosmology 
from astropy.cosmology import WMAP9 as cosmo
from astropy.cosmology import z_at_value

# Extra python functions
import HelperFunctions as func

######################################
## locations
save_loc    =  '../plots/'
data_dir    = '../output/'
# This will be put in front of the name for every figure we safe 
sim_save_str = 'N1e7_'

######################################
## PLOT setttings
plt.rc('font', family='serif')
from matplotlib import rc
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)
fsize, SMALL_SIZE, MEDIUM_SIZE, BIGGER_SIZE = 30,25,25,30
for obj in ['axes','xtick','ytick']:
    plt.rc(obj, labelsize=MEDIUM_SIZE)          # controls default text sizes
for obj in ['figure','axes']:
    plt.rc(obj, titlesize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize


######################################
## Widescreen jupyter notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Read your data

the function `read_data` is part of `HelperFunctions.py` and reads the hdf5 file containing the BBH population data and merger rates 

The Bool "DCO_mask" filters for BBHs:  
1. with an inspiral time that is less than the age of the Universe
2. excludes systems that experienced a CE from a HG donor (i.e. the flag `Optimistic_CE == False`)
3. excludes systems that experienced RLOF immediately following a CE (i.e. the flag `Immediate_RLOF>CE == False`)

In other words, we treat 2. and 3. as stellar mergers and exclude them from the rest of our analysis

## & Filter your data
Select merging BBHs using the `DCO_mask`, and aditionally exclude systems that evolve Chemically homogeneous


In [3]:
####################################################
## Reading file ##
rate_key      = 'Rates_mu00.025_muz-0.05_alpha-1.77_sigma01.125_sigmaz0.05_zBinned'
File_location = data_dir+'/COMPAS_Output_wWeights.h5'
print(File_location)

####################################################
DCO, DCO_mask, redshifts, Average_SF_mass_needed, intrinsic_rate_density, intrinsic_rate_density_z0, = func.read_data(loc = File_location, rate_key = rate_key)

DCO.info()

##########################################################
# Select merging BBHs w.o. CHE only
##########################################################
nonCHE_bool         = DCO['Stellar_Type@ZAMS(1)'] != 16
rate_nonCHE_bool    = DCO['Stellar_Type@ZAMS(1)'][DCO_mask] != 16

# Filter both the BBH table and the intrinsic rate data
merging_BBH         = DCO[DCO_mask * nonCHE_bool]
Red_intr_rate_dens  = intrinsic_rate_density[rate_nonCHE_bool, :]

print(np.shape(Red_intr_rate_dens), np.shape(merging_BBH))


../output//COMPAS_Output_wWeights.h5
<Table length=4342215>
        name          dtype 
-------------------- -------
      CE_Event_Count  uint32
    Coalescence_Time float64
    Eccentricity@DCO float64
   Immediate_RLOF>CE   uint8
    MT_Donor_Hist(1) bytes17
    MT_Donor_Hist(2) bytes17
             Mass(1) float64
             Mass(2) float64
  Merges_Hubble_Time   uint8
 Metallicity@ZAMS(1) float64
       Optimistic_CE   uint8
      Recycled_NS(1)   uint8
      Recycled_NS(2)   uint8
                SEED  uint64
   SemiMajorAxis@DCO float64
     Stellar_Type(1)   int32
     Stellar_Type(2)   int32
                Time float64
      mixture_weight float64
  SemiMajorAxis@ZAMS float64
        Mass@ZAMS(1) float64
        Mass@ZAMS(2) float64
              q_init float64
Stellar_Type@ZAMS(1)   int32
Stellar_Type@ZAMS(2)   int32
              tDelay float64
       M_moreMassive float64
       M_lessMassive float64
             q_final float64
(2252487, 200) (2252487,)


# What is the median separation at DCO formation for the CE vs the stable RLOF channel?


In [6]:
#############################################
# Calculate the median separation 
# at DCO formation ...
#############################################

# Some Bools
CE_channel   = merging_BBH['CE_Event_Count'] > 0
RLOF_channel = merging_BBH['CE_Event_Count'] == 0

print('CE systems', np.sum(merging_BBH['mixture_weight'][CE_channel]) )
print('stable RLOF systems', np.sum(merging_BBH['mixture_weight'][RLOF_channel]) )

#############################################
# Helper function to get the weighted median
#############################################
def weighted_percentile(data, percents, weights=None):
    ''' percents in units of 1%
        weights specifies the frequency (count) of data.
    '''
    if weights is None:
        return np.percentile(data, percents)
    ind=np.argsort(data)
    d=data[ind]
    w=weights[ind]
    p=1.*w.cumsum()/w.sum()*100
    y=np.interp(percents, p, d)
    return y

#############################################
# ... for BBHs that merge at z=0
#############################################
merging_BBH['SemiMajorAxis@DCO_Rsun'] = merging_BBH['SemiMajorAxis@DCO']*u.AU.to(u.Rsun)

reduced_weight_z0  = intrinsic_rate_density_z0[rate_nonCHE_bool]
median_a_all_z0    = weighted_percentile(merging_BBH['SemiMajorAxis@DCO_Rsun'], 50, weights=reduced_weight_z0)
median_a_CE_z0     = weighted_percentile(merging_BBH['SemiMajorAxis@DCO_Rsun'][CE_channel], 50, weights=reduced_weight_z0[CE_channel])
median_a_stable_z0 = weighted_percentile(merging_BBH['SemiMajorAxis@DCO_Rsun'][RLOF_channel], 50, weights=reduced_weight_z0[RLOF_channel])
    
print('\nWeighted median separation at DCO formation, for BBH merging at z=0: {} Rsun'.format(median_a_all_z0))  
print('for CE channel: {} Rsun'.format(median_a_CE_z0))  
print('for stable RLOF channel: {} Rsun'.format(median_a_stable_z0))  


#############################################
# ... for BBH observed by a perfect detector
#############################################
# Calculate the volume of the fine redshift bins
fine_volumes       = cosmo.comoving_volume(redshifts).to(u.Gpc**3).value
fine_shell_volumes = np.diff(fine_volumes) #same len in z dimension as weight
# centers of redshift bins
center_z = (redshifts[:-1] + redshifts[1:])/2.

# Convert to number of BBH merging in each shell
Rate_perfect_det     = np.sum(intrinsic_rate_density[rate_nonCHE_bool,:] * fine_shell_volumes[:]* 1./(1. + center_z), axis = 1)
    
median_a_all_allz    = weighted_percentile(merging_BBH['SemiMajorAxis@DCO_Rsun'], 50, weights=Rate_perfect_det)
median_a_CE_allz     = weighted_percentile(merging_BBH['SemiMajorAxis@DCO_Rsun'][CE_channel], 50, weights=Rate_perfect_det[CE_channel])
median_a_stable_allz = weighted_percentile(merging_BBH['SemiMajorAxis@DCO_Rsun'][RLOF_channel], 50, weights=Rate_perfect_det[RLOF_channel])
    
print('\nWeighted median separation at DCO formation, for BBH as detected by a perfect detector {} Rsun'.format(median_a_all_allz))  
print('for CE channel: {} Rsun'.format(median_a_CE_allz))  
print('for stable RLOF channel: {} Rsun'.format(median_a_stable_allz))  



CE systems 33967.34662725864
stable RLOF systems 17937.32230749049

Weighted median separation at DCO formation, for BBH merging at z=0: 22.774648199932038 Rsun
for CE channel: 14.05121953572752 Rsun
for stable RLOF channel: 31.041092341716507 Rsun

Weighted median separation at DCO formation, for BBH as detected by a perfect detector 8.16800446555253 Rsun
for CE channel: 7.337725321668436 Rsun
for stable RLOF channel: 18.767923486358804 Rsun


# Count the contribution of sub-channels

In [7]:
# Some Bools
CE_channel        = merging_BBH['CE_Event_Count'] > 0
RLOF_channel      = merging_BBH['CE_Event_Count'] == 0

two_CE_subchannel = merging_BBH['CE_Event_Count'] > 1

R_perfect_det_tot   = np.sum(Rate_perfect_det)
print('Total rate observed by perfect detector', np.sum(Rate_perfect_det) )

R_perfect_det_twoCE = np.sum(Rate_perfect_det[two_CE_subchannel])
print('Rate observed by perfect detector for double CE', R_perfect_det_twoCE )
print('this constitutes a fraction: ', R_perfect_det_twoCE/R_perfect_det_tot*100, '%')

print('stable RLOF constitutes a total fraction: ', np.sum(Rate_perfect_det[RLOF_channel])/R_perfect_det_tot *100,'%' )
print('CE channel constitutes a total fraction: ', np.sum(Rate_perfect_det[CE_channel])/R_perfect_det_tot *100,'%' )


Total rate observed by perfect detector 191538.8975909716
Rate observed by perfect detector for double CE 1168.451087965356
this constitutes a fraction:  0.6100333157709644 %
stable RLOF constitutes a total fraction:  14.94371548851971 %
CE channel constitutes a total fraction:  85.05628451148034 %


## How many BHs merge with M > 18 Msun vs M < 18 Msun at different redshifts?


In [23]:
#########################################
# centers of redshif bins
center_z = (redshifts[:-1] + redshifts[1:])/2.

#Centers of your crude redshift bins
z_bin_edges = np.array([0,0.5,1,1.5,2])
center_Crude_bins = (z_bin_edges[:-1] + z_bin_edges[1:])/2. # center points

##############################
## Calculate average rate density per z-bin
crude_rate_density = get_crude_rate_density(Red_intr_rate_dens, redshifts, z_bin_edges)


i_per_crude_bin 10.0


In [24]:
high_mass_end = merging_BBH['M_moreMassive'] > 18
low_mass_end = merging_BBH['M_moreMassive'] <= 18



R_lowM_lowz = np.sum(crude_rate_density[low_mass_end, 0]) 
R_highM_lowz = np.sum(crude_rate_density[high_mass_end, 0]) 
print('Average rate density of LOW mass BH (< 18 Msun) for $0 < z < 0.5$', R_lowM_lowz)
print('Average rate density of HIGH mass BH (> 18 Msun) for $0 < z < 0.5$',R_highM_lowz )

ratio_low_redshift = R_highM_lowz/R_lowM_lowz
print('ratio high mass/low mass at low z', ratio_low_redshift)

R_lowM_higherz = np.sum(crude_rate_density[low_mass_end, 3]) 
R_highM_higherz = np.sum(crude_rate_density[high_mass_end, 3]) 
print('Average rate density of LOW mass BH (< 18 Msun) for $1.5 < z < 2.0$', R_lowM_higherz)
print('Average rate density of HIGH mass BH (> 18 Msun) for $1.5 < z < 2.0$',R_highM_higherz )

ratio_high_redshift = R_highM_higherz/R_lowM_higherz
print('ratio high mass/low mass at high z', ratio_high_redshift)


print('ratio_high_redshift/ratio_low_redshift', ratio_low_redshift/ratio_high_redshift)



Average rate density of LOW mass BH (< 18 Msun) for $0 < z < 0.5$ 64.48739733653538
Average rate density of HIGH mass BH (> 18 Msun) for $0 < z < 0.5$ 44.34506066901206
ratio high mass/low mass at low z 0.6876546813882397
Average rate density of LOW mass BH (< 18 Msun) for $1.5 < z < 2.0$ 207.1415573901734
Average rate density of HIGH mass BH (> 18 Msun) for $1.5 < z < 2.0$ 93.42241751056946
ratio high mass/low mass at high z 0.4510076041119952
ratio_high_redshift/ratio_low_redshift 1.5247075107351844
